In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
from nba_api.stats.endpoints import leaguegamefinder, leaguedashteamstats
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Team abbreviation to full name mapping
team_abbreviation_to_full_name = {
    'ATL': 'Atlanta Hawks', 'BOS': 'Boston Celtics', 'BKN': 'Brooklyn Nets',
    'CHA': 'Charlotte Hornets', 'CHI': 'Chicago Bulls', 'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks', 'DEN': 'Denver Nuggets', 'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors', 'HOU': 'Houston Rockets', 'IND': 'Indiana Pacers',
    'LAC': 'LA Clippers', 'LAL': 'Los Angeles Lakers', 'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat', 'MIL': 'Milwaukee Bucks', 'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans', 'NYK': 'New York Knicks', 'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic', 'PHI': 'Philadelphia 76ers', 'PHX': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers', 'SAC': 'Sacramento Kings', 'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors', 'UTA': 'Utah Jazz', 'WAS': 'Washington Wizards'
}

# Reverse mapping
team_name_to_abbreviation = {v: k for k, v in team_abbreviation_to_full_name.items()}

# Standardize team names for NBA API
def standardize_team_name(team_name):
    api_name_mapping = {
        'LA Clippers': 'LA Clippers',
        'Los Angeles Clippers': 'LA Clippers',
        'Philadelphia 76ers': 'Philadelphia 76ers',
        'Sixers': 'Philadelphia 76ers',
    }
    return api_name_mapping.get(team_name, team_name)

# Determine the current NBA season
def get_current_season():
    today = datetime.today()
    year = today.year
    if today.month >= 10:  # NBA season starts in October
        return f"{year}-{str(year + 1)[2:]}"
    else:
        return f"{year - 1}-{str(year)[2:]}"

# Fetch upcoming games
def fetch_upcoming_games():
    try:
        current_season = get_current_season()
        gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=current_season)
        games_df = gamefinder.get_data_frames()[0]
        games_df['GAME_DATE'] = pd.to_datetime(games_df['GAME_DATE'])
        current_date = pd.to_datetime('today')
        upcoming_games = games_df[games_df['GAME_DATE'] >= current_date]
        upcoming_games = upcoming_games.sort_values(by='GAME_DATE', ascending=True)
        return upcoming_games[['GAME_DATE', 'MATCHUP', 'TEAM_NAME', 'GAME_ID']]
    except Exception as e:
        print(f"Error fetching games: {e}")
        return pd.DataFrame()

# Collect team stats with additional metrics
def get_team_stats():
    try:
        current_season = get_current_season()
        # Fetch advanced stats
        team_stats_advanced = leaguedashteamstats.LeagueDashTeamStats(
            measure_type_detailed_defense="Advanced", season=current_season)
        df_advanced = team_stats_advanced.get_data_frames()[0]
        
        # Fetch four factors stats
        team_stats_four_factors = leaguedashteamstats.LeagueDashTeamStats(
            measure_type_detailed_defense="Four Factors", season=current_season)
        df_four_factors = team_stats_four_factors.get_data_frames()[0]
        
        # Define expected columns
        advanced_cols = ['TEAM_NAME', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'PACE']
        four_factors_cols = ['TEAM_NAME', 'EFG_PCT', 'TM_TOV_PCT', 'OREB_PCT', 'FTA_RATE']
        
        # Check available columns
        advanced_available = [col for col in advanced_cols if col in df_advanced.columns]
        four_factors_available = [col for col in four_factors_cols if col in df_four_factors.columns]
        
        if not advanced_available or not four_factors_available:
            print("Error: Required columns missing from API response.")
            return pd.DataFrame()
        
        # Select only available columns
        df_advanced = df_advanced[advanced_available]
        df_four_factors = df_four_factors[four_factors_available]
        
        # Rename TM_TOV_PCT to TOV_PCT if present
        if 'TM_TOV_PCT' in df_four_factors.columns:
            df_four_factors = df_four_factors.rename(columns={'TM_TOV_PCT': 'TOV_PCT'})
        
        # Merge datasets
        df = pd.merge(
            df_advanced,
            df_four_factors,
            on='TEAM_NAME',
            how='inner'
        )
        
        # Standardize team names
        df['TEAM_NAME'] = df['TEAM_NAME'].apply(standardize_team_name)
        
        # Verify required columns
        required_cols = ['TEAM_NAME', 'OFF_RATING', 'DEF_RATING', 'NET_RATING']
        if not all(col in df.columns for col in required_cols):
            print("Error: Missing required columns after merge.")
            return pd.DataFrame()
        
        return df
    except Exception as e:
        print(f"Error fetching stats: {e}")
        return pd.DataFrame()

# Fetch historical game data for training
def fetch_historical_game_data(season=None):
    if season is None:
        season = get_current_season()
    try:
        gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season)
        games_df = gamefinder.get_data_frames()[0]
        games_df['GAME_DATE'] = pd.to_datetime(games_df['GAME_DATE'])
        # Filter past games
        past_games = games_df[games_df['GAME_DATE'] < pd.to_datetime('today')]
        return past_games
    except Exception as e:
        print(f"Error fetching historical games: {e}")
        return pd.DataFrame()

# Prepare training data
def prepare_training_data(seasonal_stats_df, historical_games_df):
    if seasonal_stats_df.empty:
        return pd.DataFrame()
    
    training_data = []
    metrics = ['OFF_RATING', 'DEF_RATING', 'NET_RATING']
    optional_metrics = ['PACE', 'EFG_PCT', 'TOV_PCT', 'OREB_PCT', 'FTA_RATE']
    metrics.extend([m for m in optional_metrics if m in seasonal_stats_df.columns])
    
    # Group games to avoid duplicates (home vs away)
    games = historical_games_df.groupby('GAME_ID')
    
    for game_id, game in games:
        if len(game) != 2:  # Ensure both teams are present
            continue
        team_a = game.iloc[0]
        team_b = game.iloc[1]
        
        team_a_name = standardize_team_name(team_a['TEAM_NAME'])
        team_b_name = standardize_team_name(team_b['TEAM_NAME'])
        
        # Skip if teams not in stats
        if team_a_name not in seasonal_stats_df['TEAM_NAME'].values or team_b_name not in seasonal_stats_df['TEAM_NAME'].values:
            continue
        
        team_a_stats = seasonal_stats_df[seasonal_stats_df['TEAM_NAME'] == team_a_name].iloc[0]
        team_b_stats = seasonal_stats_df[seasonal_stats_df['TEAM_NAME'] == team_b_name].iloc[0]
        
        features = {}
        for metric in metrics:
            features[f'{metric}_DIFF'] = team_a_stats[metric] - team_b_stats[metric]
        
        # Outcome: 1 if team_a won, 0 otherwise
        outcome = 1 if team_a['WL'] == 'W' else 0
        features['OUTCOME'] = outcome
        training_data.append(features)
    
    return pd.DataFrame(training_data)

# Predict game outcome with probability
def predict_game_outcome(team_a, team_b, seasonal_stats_df, model=None):
    if seasonal_stats_df.empty:
        return "Error: Team stats not available.", None
    
    team_a = standardize_team_name(team_a)
    team_b = standardize_team_name(team_b)

    if team_a not in seasonal_stats_df['TEAM_NAME'].values:
        return f"Error: {team_a} not found in stats.", None
    if team_b not in seasonal_stats_df['TEAM_NAME'].values:
        return f"Error: {team_b} not found in stats.", None

    team_a_stats = seasonal_stats_df[seasonal_stats_df['TEAM_NAME'] == team_a].iloc[0]
    team_b_stats = seasonal_stats_df[seasonal_stats_df['TEAM_NAME'] == team_b].iloc[0]

    metrics = ['OFF_RATING', 'DEF_RATING', 'NET_RATING']
    optional_metrics = ['PACE', 'EFG_PCT', 'TOV_PCT', 'OREB_PCT', 'FTA_RATE']
    metrics.extend([m for m in optional_metrics if m in seasonal_stats_df.columns])

    features = {}
    for metric in metrics:
        features[f'{metric}_DIFF'] = team_a_stats[metric] - team_b_stats[metric]

    features_df = pd.DataFrame([features])

    if model is None:
        # Fetch historical data and train model
        historical_games = fetch_historical_game_data()
        if historical_games.empty:
            return "Error: No historical game data available.", None
        training_data = prepare_training_data(seasonal_stats_df, historical_games)
        if training_data.empty:
            return "Error: No training data available.", None
        
        X_train = training_data[[f'{metric}_DIFF' for metric in metrics]]
        y_train = training_data['OUTCOME']
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

    X_test = features_df[[f'{metric}_DIFF' for metric in metrics]]
    predicted = model.predict(X_test)[0]
    prob = model.predict_proba(X_test)[0][predicted] * 100  # Probability of predicted class

    winner = team_a if predicted == 1 else team_b
    return f"{winner} wins! 🏆 ({prob:.1f}% confidence)", model

# GUI Application
class NBAPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("NBA Game Predictor 🏀")
        self.root.geometry("600x700")
        self.root.configure(bg="#1e3d59")

        # Styling
        self.style = ttk.Style()
        self.style.configure("TButton", font=("Helvetica", 12), padding=10)
        self.style.configure("TLabel", font=("Helvetica", 14), background="#1e3d59", foreground="#ffffff")
        self.style.configure("TCombobox", font=("Helvetica", 12))

        # Title
        self.title_label = ttk.Label(root, text="NBA Game Predictor", font=("Helvetica", 24, "bold"))
        self.title_label.pack(pady=20)

        # Team Selection
        self.team_frame = tk.Frame(root, bg="#1e3d59")
        self.team_frame.pack(pady=10)

        ttk.Label(self.team_frame, text="Select Team A:").grid(row=0, column=0, padx=5)
        self.team_a_var = tk.StringVar()
        self.team_a_combo = ttk.Combobox(self.team_frame, textvariable=self.team_a_var,
                                        values=sorted(team_abbreviation_to_full_name.values()),
                                        state="readonly", width=25)
        self.team_a_combo.grid(row=0, column=1, padx=5)

        ttk.Label(self.team_frame, text="Select Team B:").grid(row=1, column=0, padx=5)
        self.team_b_var = tk.StringVar()
        self.team_b_combo = ttk.Combobox(self.team_frame, textvariable=self.team_b_var,
                                        values=sorted(team_abbreviation_to_full_name.values()),
                                        state="readonly", width=25)
        self.team_b_combo.grid(row=1, column=1, padx=5)

        # Predict Button
        self.predict_button = ttk.Button(root, text="Predict Game!", command=self.predict)
        self.predict_button.pack(pady=20)

        # Result Label
        self.result_label = ttk.Label(root, text="", font=("Helvetica", 16, "bold"), foreground="#ff6f61")
        self.result_label.pack(pady=10)

        # Upcoming Games
        ttk.Label(root, text="Upcoming Games").pack()
        self.games_text = scrolledtext.ScrolledText(root, height=10, width=50, font=("Helvetica", 10))
        self.games_text.pack(pady=10)
        self.games_text.config(state='disabled')

        # Load Data
        self.seasonal_stats_df = get_team_stats()
        self.upcoming_games = fetch_upcoming_games()
        self.model = None  # Initialize model as None
        self.display_upcoming_games()

    def display_upcoming_games(self):
        self.games_text.config(state='normal')
        self.games_text.delete(1.0, tk.END)
        if self.upcoming_games.empty:
            self.games_text.insert(tk.END, "No upcoming games found.")
        else:
            for _, game in self.upcoming_games.iterrows():
                matchup = game['MATCHUP']
                try:
                    team_a, team_b = matchup.split(' vs. ')
                    team_a_full = team_abbreviation_to_full_name.get(team_a, team_a)
                    team_b_full = team_abbreviation_to_full_name.get(team_b, team_b)
                    display_matchup = f"{team_a_full} vs. {team_b_full}"
                except:
                    display_matchup = matchup
                self.games_text.insert(tk.END, f"{game['GAME_DATE'].strftime('%Y-%m-%d')}: {display_matchup}\n")
        self.games_text.config(state='disabled')

    def predict(self):
        team_a = self.team_a_var.get()
        team_b = self.team_b_var.get()

        if not team_a or not team_b:
            messagebox.showwarning("Input Error", "Please select both teams!")
            return
        if team_a == team_b:
            messagebox.showwarning("Input Error", "Please select different teams!")
            return

        self.result_label.config(text="Predicting...")
        self.root.update()
        result, self.model = predict_game_outcome(team_a, team_b, self.seasonal_stats_df, self.model)
        self.result_label.config(text=result)

# Run the App
if __name__ == "__main__":
    root = tk.Tk()
    app = NBAPredictorApp(root)
    root.mainloop()